In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [ ]:
df1 = pd.read_csv("data/DECENNIALPL2020.H1-Data.csv")
df2 = pd.read_csv("data/DECENNIALPL2020.P2-Data.csv")
df3 = pd.read_csv("data/DECENNIALPL2020.P3-Data.csv")
df4 = pd.read_csv("data/DECENNIALPL2020.P4-Data.csv") 
df5 = pd.read_csv("data/DECENNIALPL2020.P5-Data.csv")

In [3]:
def df_cleaner(df):
    df_clean = df.copy()
    
    unnamed_col = df_clean.columns[-1]
    df_clean.drop(unnamed_col, axis=1, inplace=True)
    
    new_columns = df_clean.iloc[0].str.strip().str.replace('!', '').values
    df_clean.columns = new_columns
    
    df_clean = df_clean.drop([0, 1])
    df_clean = df_clean.reset_index(drop=True)
    
    return df_clean

In [4]:
df1 = df_cleaner(df1) # Housing
df2 = df_cleaner(df2) # Hispanic/Latino
df3 = df_cleaner(df3) # Race
df5 = df_cleaner(df5) # Institutionalized

In [5]:
merged_data = df1.merge(df2, on="Geography", how="outer", suffixes=('_1', '_2')) \
                      .merge(df3, on="Geography", how="outer", suffixes=('', '_3')) \
                      .merge(df5, on="Geography", how="outer", suffixes=('', '_5'))

In [6]:
important_columns = [
    'Geography',
    'Geographic Area Name_1',
    'Total:_1',
    'Total:Occupied',
    'Total:Vacant',
    'Total:Hispanic or Latino',
    'Total:Population of one race:White alone',
    'Total:Population of one race:Black or African American alone',
    'Total:Population of one race:Asian alone',
    'Total:Population of one race:American Indian and Alaska Native alone',
    'Total:Population of one race:Native Hawaiian and Other Pacific Islander alone',
    'Total:Population of one race:Some Other Race alone',
    'Total:Not Hispanic or Latino:Population of two or more races:Population of two races:',
    'Total:Institutionalized population:',
    'Total:Institutionalized population:Correctional facilities for adults',
    'Total:Institutionalized population:Juvenile facilities',
    'Total:Institutionalized population:Nursing facilities/Skilled-nursing facilities',
    'Total:Noninstitutionalized population:College/University student housing',
    'Total:Noninstitutionalized population:Military quarters',
    'Total:Noninstitutionalized population:Other noninstitutional facilities'
]

In [ ]:
df = merged_data[important_columns]
df = df.rename(columns={
    # Geographic identifiers (tract in this case i think)
    "Geography": "geo_id",
    "Geographic Area Name_1": "area_name",
    
    # Housing data
    "Total:_1": "total_houses",
    "Total:Occupied": "occupied_houses",
    "Total:Vacant": "vacant_houses",
    
    # Race/Ethnicity data
    "Total:Hispanic or Latino": "hispanic_latino",
    "Total:Population of one race:White alone": "white",
    "Total:Population of one race:Black or African American alone": "black",
    "Total:Population of one race:Asian alone": "asian",
    "Total:Population of one race:American Indian and Alaska Native alone": "american_indian_alaska_native",
    "Total:Population of one race:Native Hawaiian and Other Pacific Islander alone": "pacific_islander",
    "Total:Population of one race:Some Other Race alone": "other_race",
    "Total:Not Hispanic or Latino:Population of two or more races:Population of two races:": "multiracial",
    
    # Institutional populations
    "Total:Institutionalized population:": "total_institutionalized",
    "Total:Institutionalized population:Correctional facilities for adults": "prison_pop",
    "Total:Institutionalized population:Juvenile facilities": "juvenile_detention_pop",
    "Total:Institutionalized population:Nursing facilities/Skilled-nursing facilities": "nursing_home_pop",
    
    # Special populations (prison, military, stuff like that)
    "Total:Noninstitutionalized population:College/University student housing": "college_dorm_pop",
    "Total:Noninstitutionalized population:Military quarters": "military_housing_pop",
    "Total:Noninstitutionalized population:Other noninstitutional facilities": "other_group_quarters_pop"
})

In [8]:
df.to_csv("data/Decennial_Census.csv")